In [1]:
import torch
import nip

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

# # T=256 
# # L=12
# # V=1
# # K=2
# # tlut_bits=10
# # decode_mode='1mad'
# # incoh_proc_mode='skip'
# # viterby_batch_size=256#1024

# T=256 
# L=16
# V=2
# K=2
# tlut_bits=9
# decode_mode='quantlut_sym'
# incoh_proc_mode='skip'
# viterby_batch_size=1024

# qmodule_test = qlib.TrellisLinear(
# 			T=T, 
#             L=L, 
#             V=V, 
#             K=K,
# 			tlut_bits=tlut_bits,
#             decode_mode=decode_mode, 
#             incoh_proc_mode=incoh_proc_mode, 
#             viterby_batch_size=viterby_batch_size
# 		)

# module_to_wrap = torch.nn.Linear(4096, 128).cuda() #model.get_submodule(module_name).cuda()
# module_weight = module_to_wrap.weight.data.clone()
# # module_to_wrap.weight.data = torch.randn(4096, 4096).cuda()
# qmodule_test = qmodule_test.wrap_module(module_to_wrap)

# w_scaled = module_weight / module_weight.std()
# wq_scaled = qmodule_test.weight / module_weight.std()

# err = torch.mean((wq_scaled - w_scaled)**2, axis=-1)
# print(f'error: {err.mean().item():.3f} +- {err.std().item():.3f}')

In [2]:
model_name='Llama2-7b-hf'
#model_name='Llama-3.2-1B'

DTYPE = torch.float16
model = qlib.load_custom_llama(
	model_name=model_name, 
	device_map='cpu', 
	torch_dtype=DTYPE, 
	#use_flash_attn=True
)

LOL, Im custom!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
#cpnt_path = '/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/T256_L12_V1_K2_1mad_qtip.pth'
cpnt_path = '/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/T256_L16_V2_K2_quantlut_sym_2d_qtip.pth'
qmodel = torch.load(cpnt_path)
qmodel = qmodel.to(DEVICE)
tokenizer=qlib.load_tokenizer('Llama2-7b-hf')

/tmp/ipykernel_193187/2858948352.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  qmodel = torch.load(cpnt_path)


In [4]:
# trellis_layer = qmodel.get_submodule('model.layers.7.self_attn.q_proj')
# test_input = torch.randn(4, 4096, 4096).half().to('cuda:0')

# %%timeit
# with torch.amp.autocast('cuda', dtype=torch.float16):
#     with torch.no_grad():
#         trellis_layer(test_input)

# full 48.8 ms
# wo incoh 44.8 ms
# wo linear 26.8 ms
# wo linear and incoh 22.8 ms
# full 1mad 61.6 ms
# full 1mad_short 73.3 ms
# full 1mad_mid 62.1 ms

In [5]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=tokenizer
).get_dataloader()


with torch.amp.autocast('cuda', dtype=torch.float16):
    res = qlib.evaluate(qmodel, dataloader, print_times=25)
    print(res)

 10%|▉         | 2/21 [00:36<05:46, 18.26s/it]

53.879631382783934


 14%|█▍        | 3/21 [00:54<05:27, 18.20s/it]

53.23798870333791


 19%|█▉        | 4/21 [01:12<05:08, 18.18s/it]

...

54.77897151625316


 95%|█████████▌| 20/21 [06:03<00:18, 18.19s/it]

54.47452254610681


100%|██████████| 21/21 [06:19<00:00, 18.07s/it]

54.97963127495153
54.97963127495153
